In [ ]:
# pip install python-Levenshtein thefuzz

In [2]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from thefuzz import fuzz

In [3]:
excel_file_path = 'MASTER PUBS DATABASE.xlsx'

sheet_name = 'MASTER' 

master_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

In [4]:
master_df

,Author Salesforce Affiliation number,Type of Publication,Number,Affiliate Author (First Name),Affiliate Author (Middle),Affiliate Author (Last Name),All Authors,Publication Title,Journal or Publisher title,Forthcoming,...,RB reviewed?,Citation,Topic Tag,Region Tag,Countries,Program Tag,Abstract,Sub-tags,When collected?,Column1
0,NaN,Working Paper,1,Raffaella,NaN,Sadun,Stephen Michael Impink; Andrea Prat; Raffaella...,Communication within Firms: Evidence from CEO ...,National Bureau of Economic Research,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer - Kiki,NaN
1,NaN,Working Paper,1,Raffaella,NaN,Sadun,Martin Gaynor; Adam Sacarny; Raffaella Sadun; ...,The Anatomy of a Hospital System Merger: The P...,National Bureau of Economic Statistics,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer - Kiki,NaN
2,NaN,Working Paper,1,Raffaella,NaN,Sadun,Laura Alfaro; Nicholas Bloom; Paola Conconi; H...,Come Together: Firm Boundaries and Delegation,National Bureau of Economic Research,Yes,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer - Kiki,NaN
3,NaN,Journal Article,1,Raffaella,NaN,Sadun,Daniela Scur; Raffaella Sadun; John Van Reenen...,The World Management Survey at 18: lessons and...,Oxford Review of Economic Policy,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer - Kiki,NaN
4,NaN,Journal Article,1,Raffaella,NaN,Sadun,Evan DeFilippis; Stephen Michael Impink; Madis...,The Impact of COVID-19 on Digital Communicatio...,Humanities and Social Sciences Communications,No,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer - Kiki,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,NaN,Working Paper,130,Max,NaN,Miller,Max Miller,Foreign Influence in US Politics,Jacobs Levy Equity Management Center for Quant...,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kiki - 10/15/23 batch,NaN
2176,NaN,Working Paper,130,Max,NaN,Miller,Max Miller,Who Hedges Interest-rate Risk? Implications fo...,Jacobs Levy Equity Management Center for Quant...,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kiki - 10/15/23 batch,NaN
2177,NaN,Journal Article,131,Natalia,NaN,Garbiras-Díaz,Natalia Garbiras-Díaz; Mateo Montenegro,All Eyes on Them: A Field Experiment on Citize...,American Economic Review,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kiki - 10/15/23 batch,NaN
2178,NaN,Journal Article,131,Natalia,NaN,Garbiras-Díaz,Aila M Matanock; Natalia Garbiras-Díaz,Considering Concessions: A Survey Experiment o...,Sage Journals,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kiki - 10/15/23 batch,NaN


In [5]:
master_df.columns

Index(['Author Salesforce Affiliation number', 'Type of Publication', 'Number',
       'Affiliate Author (First Name)', 'Affiliate Author (Middle)',
       'Affiliate Author (Last Name)', 'All Authors', 'Publication Title',
       'Journal or Publisher title', 'Forthcoming', 'Volume number',
       'Issue number', 'Page numbers', 'Quarter/Month',
       'Working paper number (if applicable)', 'Book title (if applicable)',
       'Year', 'Link', 'Note', 'Note2', 'Unsure/question', 'RB reviewed?',
       'Citation', 'Topic Tag', 'Region Tag', 'Countries', 'Program Tag',
       'Abstract', 'Sub-tags', 'When collected?', 'Column1'],
      dtype='object')

In [6]:

# [['Type of Publication', 'Number','All Authors', 'Publication Title',
#        'Journal or Publisher title', 'Forthcoming', 'Volume number',
#        'Issue number', 'Page numbers', 'Quarter/Month',
#        'Working paper number (if applicable)', 'Book title (if applicable)',
#        'Year', 'Link', 'Note', 'Note2', 'Unsure/question', 'RB reviewed?',
#        'Citation', 'Topic Tag', 'Region Tag', 'Countries', 'Program Tag',
#        'Abstract', 'Sub-tags', 'When collected?', 'Column1']]

# Bandiera, Oriana, Andrea Prat, Renata Lemos and Raffaella Sadun. 2018. Managing the Family Firm: Evidence from CEOs at Work. Review of Financial Studies 31, no. 5 (May): 1605-1653.

# All Authors | Journal/Publisher Corrected | Publication Date (Text) | Publication Title | Journal or Publisher (Publication Series Title) | Volume number | Issue number | (Month) : Page numbers

In [7]:
subset = master_df['Link'].head(20)
shape = subset.shape
shape

(20,)

In [14]:
url_list = master_df['Link'].head(20).tolist()

In [12]:
import requests
from bs4 import BeautifulSoup
import re

In [13]:
def extract_abstract_from_html(html):
    # Find the "Abstract" header text
    abstract_header = None
    soup = BeautifulSoup(html, 'html.parser')
    headers = soup.find_all(re.compile('^h[1-6]$'))
    for header in headers:
        if header.get_text(strip=True).lower() == "abstract":
            abstract_header = header
            break

    if abstract_header:
        abstract_content = ""
        next_element = abstract_header.find_next_sibling()
        while next_element and next_element.name != "h1":
            abstract_content += next_element.get_text(strip=True) + " "
            next_element = next_element.find_next_sibling()
        return abstract_content.strip()
    else:
        return "Abstract not found on the page."

def get_abstract_or_error(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        # Extract abstract text content from the HTML
        abstract_text = extract_abstract_from_html(response.content)
        return abstract_text

    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

In [15]:
abstract_texts = []
for url in url_list:
    abstract_text = get_abstract_or_error(url)
    abstract_texts.append(abstract_text)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
